In [57]:
import os
import re
import json
import langchain

--- Result of a Processed Document (Index 4) ---
Page Content:
optics for microscopy a. wave nature of light, birefringence, interference, diffraction b. optical elements: lenses, filters, polarizers c. lens properties: numerical aperture, magnification, aberrations, corrections d. resolution: rayleigh criterion, airy disc, abbe’s principle 2. light-matter interaction a. lasers b. fluorescence 3. microscope design, function and usage a. image formation in a microscope : kohler illumination, critical illumination b. brightfield, darkfield microscopy c. phase contrast microscopy d. differential interference contrast microscopy e. polarization microscopy f. epifluorescence (widefield) microscopy g. total internal reflection fluorescence (tirf) microscopy h. confocal microscopy i. super resolution microscopy 4. image detection a. camera : image capture by ccd devices b. pmts c. gaasp and hybrid silicon detectors 5. image analysis a. intensity quantification b. micrometry c. video tracking 

In [81]:
print(len(docs))

185


In [84]:
from langchain_community.document_loaders import JSONLoader
loader = JSONLoader(
    file_path='temp.json',
    jq_schema='.[]',
    text_content=False
)

docs = loader.load()


for index,doc in enumerate(docs):
    temp = json.loads(doc.page_content)
    data = ""
    del doc.metadata['source']
    del doc.metadata['seq_num']    


    for key,value in temp.items():
        if(key != "description" and key != "course_link"):
            if isinstance(value, list):
                # Join string representations of list items with spaces
                value = ' '.join(str(v) for v in value)
            doc.metadata[key] = value 
    
    for key, value in temp.items():
        if key == "description":
            if value is not None:
                # Replace newlines with spaces for uniformity
                desc = value.replace('\n', ' ')
                # Remove unnecessary '*' and '-' characters
                desc = re.sub(r'[\*\-]', '', desc)
                # Remove extra spaces
                desc = re.sub(r'\s+', ' ', desc).strip()
                # Split on bullet points (•)
                bullet_parts = [part.strip() for part in re.split(r'•', desc) if part.strip()]
                sentences = []
                for part in bullet_parts:
                    # Instead of splitting on every period, only split on periods that are not part of ordered lists (e.g., "a.", "1.")
                    # We'll use a regex to split on periods that are NOT preceded by a single letter/number and a space
                    # This will keep "a. " or "1. " together
                    sub_sentences = re.split(r'(?<!\b[a-zA-Z0-9])\.(?!\d)', part)
                    for s in sub_sentences:
                        s = s.strip()
                        if s:
                            sentences.append(s)
                # Remove extra spaces from each sentence
                sentences = [re.sub(r'\s+', ' ', s).strip() for s in sentences]
                # Reconstruct with each bullet/sentence on a new line, ending with a period
                updated_description = '.\n'.join(sentences)
                if not updated_description.endswith('.'):
                    updated_description += '.'
                data += f"{key} : {updated_description}"
            else:
                data += f"{key} : "
    del doc.metadata["class_time"]
    doc.metadata["index"]: index
    doc.page_content = data
    


In [85]:
print(docs[4])

page_content='description : 1. Optics for Microscopy a. Wave Nature of Light, Birefringence, Interference, Diffraction b. Optical elements: Lenses, Filters, Polarizers c. Lens Properties: Numerical Aperture, Magnification, Aberrations, Corrections d. Resolution: Rayleigh Criterion, Airy Disc, Abbe’s Principle 2. LightMatter Interaction a. Lasers b. Fluorescence 3. Microscope Design, Function and Usage a. Image Formation in a Microscope : Kohler Illumination, Critical illumination b. Brightfield, Darkfield Microscopy c. Phase Contrast Microscopy d. Differential Interference Contrast Microscopy e. Polarization Microscopy f. Epifluorescence (Widefield) Microscopy g. Total Internal Reflection Fluorescence (TIRF) microscopy h. Confocal Microscopy i. Super Resolution Microscopy 4. Image Detection a. Camera : Image capture by CCD devices b. PMTs c. GaAsp and Hybrid Silicon detectors 5. Image Analysis a. Intensity quantification b. Micrometry c. Video Tracking Algorithms 6. Manipulating Molecu

In [61]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
)

In [62]:
vector_store.delete_collection()

In [63]:
from langchain.vectorstores import Chroma

vector_store = Chroma(
    embedding_function = embedding_model,
    persist_directory='course_vector_database',
    collection_name = 'sample'
)

In [64]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 150, chunk_overlap=0)
chunks = text_splitter.split_documents(docs)

In [66]:
print(docs[4])

page_content='description : 1. Optics for Microscopy a. Wave Nature of Light, Birefringence, Interference, Diffraction b. Optical elements: Lenses, Filters, Polarizers c. Lens Properties: Numerical Aperture, Magnification, Aberrations, Corrections d. Resolution: Rayleigh Criterion, Airy Disc, Abbe’s Principle 2. LightMatter Interaction a. Lasers b. Fluorescence 3. Microscope Design, Function and Usage a. Image Formation in a Microscope : Kohler Illumination, Critical illumination b. Brightfield, Darkfield Microscopy c. Phase Contrast Microscopy d. Differential Interference Contrast Microscopy e. Polarization Microscopy f. Epifluorescence (Widefield) Microscopy g. Total Internal Reflection Fluorescence (TIRF) microscopy h. Confocal Microscopy i. Super Resolution Microscopy 4. Image Detection a. Camera : Image capture by CCD devices b. PMTs c. GaAsp and Hybrid Silicon detectors 5. Image Analysis a. Intensity quantification b. Micrometry c. Video Tracking Algorithms 6. Manipulating Molecu

In [67]:
len(chunks)

1178

In [68]:
vector_store.add_documents(chunks)

['f3911dfd-7352-4f05-997e-34c80bcd030a',
 'e1055546-6ed4-4c6e-b75d-77a000642b68',
 '056ef5b1-ac6e-4a7a-b7e7-d162314e9cf2',
 '38d6eb8e-82f4-411e-a0ba-f40856e1c0a7',
 '59c457fc-4221-424f-b992-6102dd8230f9',
 '89a60c08-04e4-40b3-ae44-8d60205eb075',
 '16a405c5-e40f-47b1-95c8-4f34bdc8d82e',
 '9c1f781e-8cc7-4cd9-b8e6-46ec24f4eb11',
 'ff6ae30d-6581-4027-9fa9-1828c1758db7',
 'eb2f4661-33ba-4094-8eb3-19623af54f71',
 '6e8ed6a8-59bb-4b01-a8af-08c73dc10cba',
 'bf2cf847-2b18-4f46-a641-7342a09db4ec',
 '9ee39f46-799b-4627-aa4a-1872f9002126',
 '8dff5309-215c-4989-afba-b0bc2768077f',
 '3940206f-4eb0-4327-8c62-d624697383b8',
 '7db5eb3f-60cc-4a3b-a856-6b0c3a3df1a3',
 'c5c48b6e-4ea1-4a0f-b169-7adacedb359d',
 '7e68365f-c14a-43e0-8d27-687375fc9dec',
 '897ef988-845f-4f65-8531-b08752c2a56f',
 '2d66d8e7-7232-4265-a143-f4af2044831a',
 '9d942b24-3209-409c-acca-49bdc2e60dc3',
 '0b9eebe1-5ce6-4fbf-89d0-0574171a202c',
 '611dc1ba-0493-4639-a6a8-2213765a5440',
 '66d2e6f2-01dd-4bff-b60e-872589234ede',
 'bf28c4d0-b8a2-

In [69]:
query = "Is there any course taught by Roop Mallik in slot 6?"

# CORRECTED: The key 'instructors' now matches the document metadata
filter = {
    "$and": [
        {"instructors": {"$eq": "Roop Mallik"}},
        {"slot": {"$eq": "6"}}
    ]
}
# This part is correct
retriever = vector_store.as_retriever(search_kwargs={"filter": filter})

# CORRECTED: Pass the original string query here, not the filter dictionary
results = retriever.get_relevant_documents(query)

# This part is correct
for result in results:
    print(result, '\n')

page_content='Polarizers c. Lens Properties: Numerical Aperture, Magnification, Aberrations, Corrections d. Resolution: Rayleigh Criterion, Airy Disc, Abbe’s' metadata={'duration': 'FullSemester', 'course_code': 'BB706', 'tags': 'Theory', 'credits': 6, 'slot': '6', 'course_name': 'Fundamental And Practical Aspects Of Microscopy', 'venue': 'Class Room : LT 104', 'instructors': 'Roop Mallik', 'prerequisites': '', 'department': 'Biosciences & Bioengineering', 'is_running': True} 

page_content='Principle 2. LightMatter Interaction a. Lasers b. Fluorescence 3. Microscope Design, Function and Usage a. Image Formation in a Microscope : Kohler' metadata={'is_running': True, 'course_name': 'Fundamental And Practical Aspects Of Microscopy', 'department': 'Biosciences & Bioengineering', 'slot': '6', 'credits': 6, 'duration': 'FullSemester', 'course_code': 'BB706', 'instructors': 'Roop Mallik', 'tags': 'Theory', 'venue': 'Class Room : LT 104', 'prerequisites': ''} 

page_content='Illumination, Cr

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

# --- 1. Setup Data and Vector Store ---
# This setup assumes you don't have an existing vector_store variable.
# If you do, you can skip this part.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
course_doc = Document(
    page_content="This course offers a comprehensive look at modern microscopy...",
    metadata={
        'course_code': 'BB706',
        'course_name': 'Fundamental And Practical Aspects Of Microscopy',
        'department': 'Biosciences & Bioengineering',
        'instructors': 'I - Roop Mallik', # Note: key is 'instructors' (plural)
        'tags': 'Theory',
        'credits': 6,
        'prerequisites': '',
        'is_running': True,
        'venue': 'Class Room : LT 104',
        'duration': 'FullSemester',
        'slot': '6'
    }
)
vector_store = Chroma.from_documents([course_doc], embedding=embeddings)


# --- 2. Manually Test the Metadata Filter ---

# The original user question
query = "Is there any course taught by Roop Mallik in slot 6?"

# The filter you manually created (simulating the LLM router's output)
# Note the corrected keys and values to match the metadata exactly.
manual_filter = {
    "instructors": "Roop Mallik",
    "slot": "6"
}

print(f"Testing with filter: {manual_filter}")

# Configure the retriever with your filter
retriever = vector_store.as_retriever(
    search_kwargs={"filter": manual_filter}
)

# Execute the search using the original string query
# The retriever will first apply the filter, then perform the search.
results = retriever.get_relevant_documents(query)


# --- 3. Print the Results ---
if results:
    print("\n✅ Found matching documents:")
    for result in results:
        print(result, '\n')
else:
    print("\n❌ No documents found matching the filter.")

Testing with filter: {'instructors': 'I - Roop Mallik', 'slot': '6'}


ValueError: Expected where to have exactly one operator, got {'instructors': 'I - Roop Mallik', 'slot': '6'} in query.

In [4]:
import os
from dotenv import load_dotenv

# This line loads the variables from your .env file into the environment
load_dotenv()

# You can now access your key using os.getenv()
my_api_key = os.getenv("GOOGLE_API_KEY")

# You can verify it's loaded (optional)
if my_api_key:
    print("✅ API Key loaded successfully!")
    # Your LangChain or other code that needs the key can now run.
    # It will often find the key automatically from the environment.
else:
    print("❌ Could not load API Key. Check your .env file and path.")

# Example: Using it with LangChain
# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # This automatically uses the loaded key

✅ API Key loaded successfully!


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class CourseMetaDataSearch(BaseModel):
    """
    A tool to find courses using specific metadata filters like instructor name, 
    course code, or slot number.
    """
    course_code : str = Field(default = None, description="The code for the course, e.g., 'AE103', 'CS419")
    instructors : str = Field(default = None, description="The name of the instructor teaching the course")
    slot: str = Field(default=None, description="The schedule slot for the course, e.g., '4'")


router_prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert at routing a user's query. "
            "If the query contains specific criteria like an instructor's name, "
            "course code, or slot number, extract them. Otherwise, do not extract anything."),
        ("human", "{query}")
    ]
)

gemini_router_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

llm_with_tool = gemini_router_llm.with_structured_output(CourseMetaDataSearch)
pre_retrieval_router_chain = router_prompt | llm_with_tool

In [ ]:
query = "Find me a couse offered by professor Prabhu Ramachandran."

def retrieve_documents(query : str):
    router_decision = pre_retrieval_router_chain.invoke({"query":{query}})
    
    search_filter = {key: value for key,value in router_decision.dict().items() if value is not None}

retrieve_documents(query)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

gemini_generator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.5) 